In [1]:
import pandas as pd
import numpy as np

In [2]:
app_df=pd.read_csv("../content/cleaned_application.csv",index_col=0)

In [3]:
credit_record=pd.read_csv("../content/credit_record.csv",index_col=0)

C:\Users\siong hwee\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
app_df.head()

,id,gender,own_car,own_realty,num_child,income,income_type,education_level,family_status,house_type,birth_day,employment_length,mobile,work_phone,phone,email,job,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,High skill tech staff,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,High skill tech staff,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
app_df["mobile"].unique()

array([1], dtype=int64)

Everyone has a mobile phone, we should drop this redundant factor (cardinality=1) to reduce the dimesionality of our data

In [6]:
prev_df=app_df.drop(columns=["mobile"],inplace=True)

In [7]:
credit_record

,id,month_bal,status
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [8]:
credit_record["month_bal"].unique(),credit_record["status"].unique()

(array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
        -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
        -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
        -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
        -52, -53, -54, -55, -56, -57, -58, -59, -60], dtype=int64),
 array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object))

We define a good label as a person not having >1 month of no loan payment

In [9]:
good_records=["X","0","C"]

In [13]:
label_df=credit_record.groupby("id").apply(lambda x: all(x["status"].isin(good_records)))

In [28]:
ids=pd.DataFrame({"label":label_df}).reset_index()


In [30]:
ids["label"]=ids["label"].apply(lambda x:int(x))

Data preprocessing

In [34]:
app_df["employment_length"].describe()

count    438557.000000
mean      60563.675328
std      138767.799647
min      -17531.000000
25%       -3103.000000
50%       -1467.000000
75%        -371.000000
max      365243.000000
Name: employment_length, dtype: float64

In [35]:
app_df[app_df["employment_length"]!=365243]["employment_length"].describe()

count    363228.000000
mean      -2623.052700
std        2399.514253
min      -17531.000000
25%       -3512.000000
50%       -1924.000000
75%        -915.000000
max         -12.000000
Name: employment_length, dtype: float64

In [36]:
pensioners=app_df[app_df["employment_length"]==365243]

In [37]:
pensioners["income_type"].unique()

array(['Pensioner'], dtype=object)

In [38]:
pensioners["birth_day"].describe()

count    75329.000000
mean    -21715.424272
std       1839.857354
min     -25201.000000
25%     -22981.000000
50%     -21927.000000
75%     -20807.000000
max      -8145.000000
Name: birth_day, dtype: float64

Having the pensioners with employment length = 365243 is both incorrect (1000 years) and will definitely affect the feature scaling of employment length. Thus , we propose that the retirement age is the median of the dataset (21927) and employment length will be max(median-age,0).

In [39]:
pensioners["employment_length"]=pensioners.apply(lambda x : max(-21927-x["birth_day"],0),axis=1)

C:\Users\SIONGH~1\AppData\Local\Temp/ipykernel_9304/2582114183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pensioners["employment_length"]=pensioners.apply(lambda x : max(-21927-x["birth_day"],0),axis=1)


In [40]:
app_df.update(pensioners)

In [41]:
cat_features = app_df.columns[app_df.dtypes == 'object'].to_list()

In [42]:
app_df=app_df.fillna("Pensioner")

In [43]:
app_df[cat_features].describe()

,gender,own_car,own_realty,income_type,education_level,family_status,house_type,job
count,438557,438557,438557,438557,438557,438557,438557,438557
unique,2,2,2,5,5,5,6,19
top,F,N,Y,Working,Secondary / secondary special,Married,House / apartment,Laborers
freq,294440,275459,304074,226104,301821,299828,393831,90688


We can use ordinal scaling for the factors ,even gender as there is only 2 genders and both the ordinal and nominal classification lead to 0 and 1 encoding so there is no difference. However, we have to create our own ranking for the factors with cardinality>2.

In [44]:
auto_cat_features=["gender","own_car","own_realty"]

In [45]:
for i in auto_cat_features:
    cat_features.remove(i)

In [46]:
cat_features

['income_type', 'education_level', 'family_status', 'house_type', 'job']

In [47]:
from sklearn.preprocessing import OrdinalEncoder

In [48]:
df1=app_df.copy()

In [49]:
autoenc =OrdinalEncoder()
autoenc.fit(app_df[auto_cat_features])
df1[auto_cat_features]=autoenc.transform(app_df[auto_cat_features])

In [50]:
testenc =OrdinalEncoder()
testenc.fit(app_df[cat_features])
testenc.categories_

[array(['Commercial associate', 'Pensioner', 'State servant', 'Student',
        'Working'], dtype=object),
 array(['Academic degree', 'Higher education', 'Incomplete higher',
        'Lower secondary', 'Secondary / secondary special'], dtype=object),
 array(['Civil marriage', 'Married', 'Separated', 'Single / not married',
        'Widow'], dtype=object),
 array(['Co-op apartment', 'House / apartment', 'Municipal apartment',
        'Office apartment', 'Rented apartment', 'With parents'],
       dtype=object),
 array(['Accountants', 'Cleaning staff', 'Cooking staff', 'Core staff',
        'Drivers', 'HR staff', 'High skill tech staff', 'IT staff',
        'Laborers', 'Low-skill Laborers', 'Managers', 'Medicine staff',
        'Pensioner', 'Private service staff', 'Realty agents',
        'Sales staff', 'Secretaries', 'Security staff',
        'Waiters/barmen staff'], dtype=object)]

In [51]:
categories=[['Student', 'Pensioner','Working','State servant','Commercial associate'],
     ['Lower secondary', 'Secondary / secondary special','Incomplete higher', 'Higher education', 'Academic degree'],
            ['Widow','Single / not married','Separated','Civil marriage', 'Married'],
            ['Rented apartment','With parents','Co-op apartment', 'Municipal apartment','Office apartment', 'House / apartment']]

Lets use median income level to justify the ordering of jobs instead of a biased personal heuristic!

In [52]:
job_income=dict(app_df.groupby("job").apply(lambda x: x["income"].median()))

In [53]:
sorted_job_income={k: v for k, v in sorted(job_income.items(), key=lambda item: item[1])}

In [54]:
job_ranking=list(sorted_job_income.keys())

In [55]:
categories.append(job_ranking)

In [56]:
catenc =OrdinalEncoder(categories=categories)
catenc.fit(app_df[cat_features])
df1[cat_features]=catenc.transform(app_df[cat_features])

We can normalise birthday,employment length,num_child and standarise income and CNT_FAM_MEMBERS 

In [57]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [58]:
num_features = app_df.columns[app_df.dtypes != 'object'].to_list()
num_features.remove('id')

In [59]:
app_df[num_features]

,num_child,income,birth_day,employment_length,work_phone,phone,email,CNT_FAM_MEMBERS
0,0.0,427500.0,-12005.0,-4542.0,1.0,0.0,0.0,2.0
1,0.0,427500.0,-12005.0,-4542.0,1.0,0.0,0.0,2.0
2,0.0,112500.0,-21474.0,-1134.0,0.0,0.0,0.0,2.0
3,0.0,270000.0,-19110.0,-3051.0,0.0,1.0,1.0,1.0
4,0.0,270000.0,-19110.0,-3051.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
438552,0.0,135000.0,-22717.0,790.0,0.0,0.0,0.0,1.0
438553,0.0,103500.0,-15939.0,-3007.0,0.0,0.0,0.0,1.0
438554,0.0,54000.0,-8169.0,-372.0,1.0,0.0,0.0,1.0
438555,0.0,72000.0,-21673.0,0.0,0.0,0.0,0.0,2.0


In [60]:
std_feature=["income","CNT_FAM_MEMBERS"]

In [61]:
for i in std_feature:
    num_features.remove(i)
norm_features=std_feature.copy()

In [62]:
norm=MinMaxScaler()
norm.fit(app_df[norm_features])
df1[norm_features]=norm.transform(app_df[norm_features])

In [63]:
std=StandardScaler()
std.fit(app_df[std_feature])
df1[std_feature]=std.transform(app_df[std_feature])

In [71]:
training_df=ids.merge(df1,how="inner")

In [72]:
training_df

,id,label,gender,own_car,own_realty,num_child,income,income_type,education_level,family_status,house_type,birth_day,employment_length,work_phone,phone,email,job,CNT_FAM_MEMBERS
0,5008804,0,1.0,1.0,1.0,0.0,2.179879,2.0,3.0,3.0,0.0,-12005.0,-4542.0,1.0,0.0,0.0,13.0,-0.216745
1,5008805,0,1.0,1.0,1.0,0.0,2.179879,2.0,3.0,3.0,0.0,-12005.0,-4542.0,1.0,0.0,0.0,13.0,-0.216745
2,5008806,1,1.0,1.0,1.0,0.0,-0.681502,2.0,1.0,4.0,5.0,-21474.0,-1134.0,0.0,0.0,0.0,9.0,-0.216745
3,5008808,1,0.0,0.0,1.0,0.0,0.749189,4.0,1.0,1.0,5.0,-19110.0,-3051.0,0.0,1.0,1.0,8.0,-1.331316
4,5008809,1,0.0,0.0,1.0,0.0,0.749189,4.0,1.0,1.0,5.0,-19110.0,-3051.0,0.0,1.0,1.0,8.0,-1.331316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5150482,1,0.0,1.0,1.0,1.0,-0.272733,2.0,1.0,4.0,5.0,-10808.0,-1739.0,0.0,0.0,0.0,10.0,0.897826
36453,5150483,1,0.0,1.0,1.0,1.0,-0.272733,2.0,1.0,4.0,5.0,-10808.0,-1739.0,0.0,0.0,0.0,10.0,0.897826
36454,5150484,1,0.0,1.0,1.0,1.0,-0.272733,2.0,1.0,4.0,5.0,-10808.0,-1739.0,0.0,0.0,0.0,10.0,0.897826
36455,5150485,1,0.0,1.0,1.0,1.0,-0.272733,2.0,1.0,4.0,5.0,-10808.0,-1739.0,0.0,0.0,0.0,10.0,0.897826


In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  KFold, cross_val_score
from sklearn.metrics import GridSearchCV, cross_validate, KFold, cross_val_score

In [94]:
def kf_cross_val(model, X, y): 
    scores = []
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
        x_train = X.iloc[train_index]
        y_train = y.loc[train_index]
        x_test = X.loc[test_index]
        y_test = y.loc[test_index]
        model.fit(x_train,y_train)
        y_pred = model.predict_proba(x_test)[:,1]     
        scores.append(roc_auc_score(y_test,y_pred))       
    return scores

In [95]:
X_train=training_df[training_df.columns.difference(['id','label'])]
y_train=training_df["label"].copy()

In [96]:
lr = LogisticRegression(max_iter=2000, random_state=0)
lr_score = kf_cross_val(lr, X_train, y_train)
print(f"L2 Regularization: {round(np.mean(lr_score),5)} ({round(np.std(lr_score),5)})")

lr = LogisticRegression(max_iter=2000, penalty='none', random_state=0)
lr_score = kf_cross_val(lr, X_train, y_train)
print(f"No Regularization: {round(np.mean(lr_score),5)} ({round(np.std(lr_score),5)})")

L2 Regularization: 0.5199 (0.00376)
No Regularization: 0.5199 (0.00376)


It seems logistic Regression is no better than guessing due to the ordinal features in the dataset